In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...


In [ ]:
!conda --version

conda 24.11.2


In [ ]:
!pip install biopython --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 106.9 MB/s eta 0:00:00


In [ ]:
import sys

sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
!pwd

/content


In [ ]:
! grep ">" "/content/drive/MyDrive/Proyecto/conoserver_250206_protein.fa" | wc -l

grep: /content/drive/MyDrive/Proyecto/conoserver_250206_protein.fa: No such file or directory
0


# Removinedo sinteticos y no completos

In [ ]:
!grep -v "Synthetic" "/content/drive/MyDrive/Proyecto/conoserver_250206_protein.fa" > "/content/drive/MyDrive/Proyecto/sin_synthetic_proteins.fa"

/bin/bash: line 1: /content/drive/MyDrive/Proyecto/sin_synthetic_proteins.fa: No such file or directory


In [ ]:
! awk '/^>/ {p = ($0 !~ /Synthetic/)} p' "/content/drive/MyDrive/Proyecto/conoserver_250206_protein.fa" > "/content/drive/MyDrive/Proyecto/sin_synthetic_proteins.fa"


/bin/bash: line 1: /content/drive/MyDrive/Proyecto/sin_synthetic_proteins.fa: No such file or directory


In [ ]:
! cat "/content/drive/MyDrive/Proyecto/sin_synthetic_proteins.fa"

cat: /content/drive/MyDrive/Proyecto/sin_synthetic_proteins.fa: No such file or directory


In [ ]:
!sed '/^>/!s/[Xx]//g' "/content/drive/MyDrive/Proyecto/sin_synthetic_proteins.fa" > "/content/drive/MyDrive/Proyecto/sin_X_proteins.fa"


/bin/bash: line 1: /content/drive/MyDrive/Proyecto/sin_X_proteins.fa: No such file or directory


In [ ]:
! tail -n 50 "/content/drive/MyDrive/Proyecto/sin_X_proteins.fa"

tail: cannot open '/content/drive/MyDrive/Proyecto/sin_X_proteins.fa' for reading: No such file or directory


In [ ]:
! grep -v "^>" "/content/drive/MyDrive/Proyecto/sin_X_proteins.fa" | wc -l

7280


In [ ]:
!conda install -c bioconda cd-hit -y

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cd-hit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    cd-hit-4.8.1               |      h5ca1c30_12         189 KB  bioconda
    certifi-2025.1.31          |     pyhd8ed1ab_0         159 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    openssl-3.4.1              |       h7b32b05_0         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.4 MB

The following NEW packages will be INSTALLED:

  cd-hit             bioconda/linux-64::cd-hit-4.8.1-h5ca1c30_12 

The following package

In [ ]:
! cd-hit -i "/content/drive/MyDrive/Proyecto/sin_X_proteins.fa" -o "conoserver_250206_protein_cdhit.fa" -c 0.95

Program: CD-HIT, V4.8.1 (+OpenMP), Dec 13 2024, 22:16:14
Command: cd-hit -i
         /content/drive/MyDrive/Proyecto/sin_X_proteins.fa -o
         conoserver_250206_protein_cdhit.fa -c 0.95

Started: Thu Mar 20 04:33:42 2025
                            Output                              
----------------------------------------------------------------
total seq: 7045
longest and shortest : 448 and 11
Total letters: 344527
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 1M
Buffer          : 1 X 16M = 16M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 83M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 89624965

comparing sequences from          0  to       7045
.......
     7045  finished       2832  clusters

Approximated maximum memory consumption: 84M
writing new database
writing clustering information
program completed !

Total CPU time 0.24


In [ ]:
! ls -la

total 1008
drwxr-xr-x 1 root root   4096 Mar 18 23:27 .
drwxr-xr-x 1 root root   4096 Mar 18 23:23 ..
-rw-r--r-- 1 root root  21477 Mar 18 23:26 condacolab_install.log
drwxr-xr-x 4 root root   4096 Mar 17 13:31 .config
-rw-r--r-- 1 root root 632117 Mar 18 23:27 conoserver_250206_protein_cdhit.fa
-rw-r--r-- 1 root root 349522 Mar 18 23:27 conoserver_250206_protein_cdhit.fa.clstr
drwx------ 7 root root   4096 Mar 18 23:25 drive
drwxr-xr-x 1 root root   4096 Mar 17 13:32 sample_data


In [ ]:
! grep ">" "drive/MyDrive/Proyecto/conoserver_250206_protein.fa" | wc -l

8523


In [ ]:
! grep ">" "conoserver_250206_protein_cdhit.fa" | wc -l

2832


In [ ]:
! grep ">" "conoserver_250206_protein_cdhit.fa.clstr" | wc -l

9877


In [ ]:
! head -n 50 conoserver_250206_protein_cdhit.fa.clstr

>Cluster 0
0	448aa, >P05538|conohyal-Cn1... *
1	430aa, >P05539|conohyal-Cn1... at 100.00%
>Cluster 1
0	346aa, >P05536|conohyal-ad1... *
1	326aa, >P05537|conohyal-ad1... at 100.00%
>Cluster 2
0	256aa, >P10118|Conoporin... *
1	227aa, >P10127|Conoporin... at 100.00%
>Cluster 3
0	248aa, >P05534|Conoporin-Cn... *
1	221aa, >P05535|Conoporin-Cn... at 100.00%
>Cluster 4
0	11aa, >P07534|ConoCAP-Vilc... at 100.00%
1	207aa, >P08785|ConoCAP... *
>Cluster 5
0	196aa, >P06730|prohormone-4... *
1	11aa, >P06867|prohormone-4... at 100.00%
>Cluster 6
0	192aa, >P08475|Conkunitzin-... *
>Cluster 7
0	188aa, >P06389|conodipine-V... *
1	76aa, >P06662|Conodipine-V... at 100.00%
2	53aa, >P08311|Conodipine-V... at 100.00%
>Cluster 8
0	188aa, >P06390|Conodipine-V... *
1	76aa, >P06661|Conodipine-V... at 100.00%
2	53aa, >P08310|Conodipine-V... at 100.00%
>Cluster 9
0	76aa, >P09900|Conodipine-L... at 100.00%
1	182aa, >P09901|Conodipine-L... *
2	47aa, >P10502|Conodipine-L... at 100.00%
>Cluster 10
0	181aa, >P10107|Co

In [ ]:
! tail -n 10 conoserver_250206_protein_cdhit.fa.clstr

>Cluster 2827
0	11aa, >P02221|Sequence... *
>Cluster 2828
0	11aa, >P02541|VcVB|Conus... *
>Cluster 2829
0	11aa, >P04229|Ar1311|Conus... *
>Cluster 2830
0	11aa, >P05219|Pc16a|Conus... *
>Cluster 2831
0	11aa, >P05828|Tx5.3|Conus... *


In [ ]:
from google.colab import files
files.download("conoserver_250206_protein_cdhit.fa.clstr")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from Bio.Align.Applications import MafftCommandline
from Bio import AlignIO
from Bio import SeqIO


/usr/local/lib/python3.11/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [ ]:
def extract_cluster_sequences(cluster_filename,fasta_filename,output_filename):
  with open(cluster_filename,'r') as cluster_file:
    cluster_ids = []
    for line in cluster_file:
      if line.startswith('>'):
        continue
      seq_id = line.split('>')[1].split('...')[0]
      cluster_ids.append(seq_id)

  sequences = SeqIO.parse(fasta_filename,'fasta')
  with open(output_filename,'w') as output:
    SeqIO.write((seq for seq in sequences if seq.id in cluster_ids),output, 'fasta')

extract_cluster_sequences('')


TypeError: extract_cluster_sequences() missing 2 required positional arguments: 'fasta_filename' and 'output_filename'

# Hidden Markov Model (HMM)

In [ ]:
!pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
from hmmlearn import hmm

file_fasta = "/content/drive/MyDrive/Proyecto/aligned_cluster_sequences.fasta"

V = np.array(['A','G','C','T']) # Matriz de simbolos
model = hmm.CategoricalHMM()



In [ ]:
def forward(obs_seq, states, start_prob, trans_prob, emit_prob):
    fwd = [{}]
    for state in states:
        fwd[0][state] = start_prob[state] * emit_prob[state][obs_seq[0]]
    for t in range(1, len(obs_seq)):
        fwd.append({})
        for state in states:
            fwd[t][state] = sum(fwd[t-1][prev_state] * trans_prob[prev_state][state] * emit_prob[state][obs_seq[t]]
                                for prev_state in states)
    prob = sum(fwd[-1][state] for state in states)
    return fwd, prob

def backward(obs_seq, states, start_prob, trans_prob, emit_prob):
    bkw = [{} for _ in range(len(obs_seq))]
    T = len(obs_seq)
    for state in states:
        bkw[T-1][state] = 1

    for t in reversed(range(T-1)):
        for state in states:
            bkw[t][state] = sum(bkw[t+1][next_state] * trans_prob[state][next_state] *
                                emit_prob[next_state][obs_seq[t+1]]
                                for next_state in states)
    prob = sum(start_prob[state] * emit_prob[state][obs_seq[0]] * bkw[0][state] for state in states)
    return bkw, prob

In [ ]:
def baum_welch(obs_seq, states, start_prob, trans_prob, emit_prob):
    T = len(obs_seq)
    M = len(emit_prob[states[0]])

    while True:
        fwd, _ = forward(obs_seq, states, start_prob, trans_prob, emit_prob)
        bkw, _ = backward(obs_seq, states, start_prob, trans_prob, emit_prob)

        A = np.zeros((len(states), len(states)))
        E = np.zeros((len(states), M))

        for t in range(T-1):
            denom = sum(fwd[t][state] * bkw[t][state] for state in states)
            for i, state_i in enumerate(states):
                for j, state_j in enumerate(states):
                    A[i][j] += fwd[t][state_i] * trans_prob[state_i][state_j] * emit_prob[state_j][obs_seq[t+1]] *bkw[t+1][state_j] / denom

                E[i][obs_seq[t]] += fwd[t][state_i] * bkw[t][state_i] / denom

        for i in range(len(states)):
            start_prob[states[i]] = fwd[0][states[i]] * bkw[0][states[i]] / sum(fwd[0][state] *
                                    bkw[0][state] for state in states)

        for i, state_i in enumerate(states):
            denom = sum(A[i][j] for j in range(len(states)))
            for j in range(len(states)):
                trans_prob[state_i][states[j]] = A[i][j] / denom

            denom = sum(E[i][o] for o in range(M))
            for o in range(M):
                emit_prob[state_i][o] = E[i][o] / denom

        break

    return start_prob, trans_prob, emit_prob

In [ ]:
states = np.arra(['M','I','D'])
start_prob = np.array([0.1,0.5,0.4])

obs_seq = np.zeros([])
trans_prob = np.zeros([])
emit_prob = np.zeros([])

start_prob, trans_prob, emit_prob = baum_welch(obs_seq, states, start_prob, trans_prob, emit_prob)

## David


In [ ]:
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio import AlignIO
from Bio import SeqIO
import os
import subprocess
from collections import defaultdict
import pprint

In [ ]:
def extract_cluster_sequences(cluster_filename,fasta_filename,output_filename):
    with open(cluster_filename,'r',encoding='utf-8') as cluster_file:
        cluster_ids = []
        for line in cluster_file:
            if line.startswith('>'):
                continue
            if(line.__contains__("*")):
                seq_id = line.split('>')[1].split('...')[0]
            cluster_ids.append(seq_id)
    with open(fasta_filename, 'r', encoding='utf-8') as fasta_file:
        sequences = SeqIO.parse(fasta_file, 'fasta')
        filtered_seqs = [seq for seq in sequences if seq.id in cluster_ids]
    with open(output_filename, 'w', encoding='utf-8') as output:
        SeqIO.write(filtered_seqs, output, 'fasta')

In [ ]:
def read_sequences(align_filename):
    alignment = []
    with open(align_filename,'r',encoding='utf-8') as align_file:
        current_line = ""
        for line in align_file:
            if line.startswith('>'):
                alignment.append(current_line)
                current_line =''
                continue
            else:
                current_line+=line
                current_line=current_line.replace('\n',"")
    return alignment

extract_cluster_sequences(
    cluster_filename='./BioInf_ProyectoFinal/conoserver_250206_protein_cdhit.fa.clstr',
    fasta_filename='./BioInf_ProyectoFinal/conoserver_250206_protein.fa',
    output_filename='./BioInf_ProyectoFinal/cluster_sequences.fasta'
)

In [ ]:
def identify_match_columns(alignment, threshold=0.5):
    nseq = len(alignment)
    ncol = len(alignment[1])
    match_cols =[]
    for j in range(ncol):
        count_non_gap = sum(1 for seq in alignment if seq[j] != '-')
        if count_non_gap / nseq >= threshold:
            match_cols.append(j)
    return match_cols

In [ ]:
def get_state_path(seq,match_cols):
    state_path =['S']
    match_index = 0
    last_match_index = None
    ncol = len(seq)
    for j in range(ncol):
        if j in match_cols:
            match_index +=1
            if seq[j] =='-':
                state_path.append(f'D{match_index}')
            else:
                state_path.append(f'M{match_index}')
            last_match_index = match_index
        else:
            # Columna de insercion
            if seq[j] !='-':
                # El estado de incersion se asocia al ultimo match (o 0 si no hay match previo)
                ins_state = f'I{last_match_index if last_match_index is not None else 0}'
                state_path.append(ins_state)
    state_path.append('E')
    return state_path

In [ ]:
def calculate_transitions(state_paths):
    transition_counts = defaultdict(int)
    for path in state_paths:
        for i in range(len(path)-1):
            transition = (path[i],path[i+1])
            transition_counts[transition]+=1
    return transition_counts

In [ ]:
def get_emissions(seq, match_cols):
    emissions = []
    match_index = 0
    last_match_index = None
    ncol = len(seq)
    for j in range(ncol):
        if j in match_cols:
            match_index +=1
            if seq[j] != '-':
                state = f'M{match_index}'
                emissions.append((state,seq[j]))
            # Si es gap, se considera delecion (no emite)
            last_match_index = match_index
        else:
            # Columna de insercion: si hay residuo, emite
            if seq[j] !='-':
                ins_state=f'I{last_match_index if last_match_index is not None else 0}'
                emissions.append((ins_state,seq[j]))
    return emissions


In [ ]:
alignment = read_sequences('./BioInf_ProyectoFinal/aligned_cluster_sequences_01.fasta')
alignment.pop(0)

match_cols = identify_match_columns(alignment, threshold=0.5)
# Generar la trayectoria de estados para cada secuencia
state_paths = [get_state_path(seq,match_cols) for seq in alignment]
for i, path in enumerate(state_paths):
    print(f'Secuencia {i+1} - trayectoria de estados: {path}')

# Generar las transiciones entre estados
transition_counts = calculate_transitions(state_paths)
for trans,count in transition_counts.items():
    print(f"{trans}: {count}")

# Generar matriz de emisiones
emission_counts = defaultdict(lambda:defaultdict(int))
for seq in alignment:
    for state, symbol in get_emissions(seq,match_cols):
        emission_counts[state][symbol]+=1

print("\nContabilización de Emisiones (por estado):")
for state, counts in emission_counts.items():
    print(f"{state}: {dict(counts)}")

# Calcular las probabilidades de emision y transicion (usando pseudoconteos)
alpha= 1
protein_alphabet = list("ARNDCEQGHILKMFPSTWYVUOBZXJ")
emission_probabilities = {}
for state,counts in emission_counts.items():
    total = sum(counts.values())+ alpha * len(protein_alphabet)
    emission_probabilities[state]={}
    for letter in protein_alphabet:
        emission_probabilities[state][letter] = (counts.get(letter,0)+alpha)/total

print("\nProbabilidades de Emisión (con pseudocontadores):")
pprint.pprint(emission_probabilities)

alpha_trans = 1  # Pseudocontador para transiciones
from_state_transitions = defaultdict(lambda:defaultdict(int))
for(from_state,to_state), count in transition_counts.items():
    from_state_transitions[from_state][to_state] =count

transition_probabilities = {}
for from_state,transitions in from_state_transitions.items():
    # Numero de transiciones posibles observadas desde 'from_state'
    total = sum(transitions.values())+alpha_trans *len(transitions)
    transition_probabilities[from_state]={}
    for to_state,count in transitions.items():
        transition_probabilities[from_state][to_state] = (count+alpha_trans)/total

print("\nProbabilidades de Transición (con pseudocontadores):")
pprint.pprint(dict(transition_probabilities))